Install HPI tool and get trainings-data, init keras and prepare data

In [6]:
# Befehl für Google Colab für Tensorflow 2
%tensorflow_version 2.x 

# ohne Ausrufezeichen bei Ausführung im lokalen Notebook
!pip install --upgrade deeplearning2020
from deeplearning2020.datasets import ImageWoof

#!pip install tf-nightly-gpu

train_data, test_data, classes = ImageWoof.load_data()

# TensorFlow ≥2.0 wird benötigt
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.test.is_gpu_available():
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
else:
  print("GPU ok")

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)


import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Activation, Input, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from scipy.stats import reciprocal
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

!pip install --upgrade deeplearning2020
from deeplearning2020 import helpers

# jupyters notebook Befehl zum direkten Anzeigen von Matplotlib Diagrammen
%matplotlib inline

#get helpers
from deeplearning2020 import helpers

#show sample data
#helpers.plot_images(train_data_orig.take(9), classes)

#def preprocess(image, label):
#    resized_image = tf.image.resize(image, [300, 300])
#    return resized_image, label

## Festlegung der Batch Größe für die Datenvorbereitung
#batch_size = 32 

## Durchmischen der Trainingsdaten, dass nicht mit sortierten Bildern trainiert wird 
#train_data = train_data.shuffle(1000) 
#
#print('shape des Trainigsdatensatzes vor dem preprocessing: ', train_data)
#
#train_data = train_data.map(preprocess) \
#  .batch(batch_size).prefetch(1)          
#test_data = test_data.map(preprocess) \
#  .batch(batch_size).prefetch(1)

#print('shape des Traingingsdatensatzes nach dem preprocessing: ', train_data)

#show refined data
#helpers.plot_images(train_data.unbatch().take(9), classes)



Requirement already up-to-date: deeplearning2020 in /usr/local/lib/python3.6/dist-packages (0.4.21)
/root/.keras/datasets/imagewoof2-320/train
Loaded 9025 images
/root/.keras/datasets/imagewoof2-320/val
Loaded 3929 images
GPU ok
Sun Apr 12 21:43:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    32W / 250W |   1395MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                   

In [0]:
img_sizeX = 300
img_sizeY = 300

n_train = 9025
n_test = 3929

import typing
from typing import TYPE_CHECKING

def myPreprocess(image, label):
  resized_image = tf.image.resize(image, [img_sizeX, img_sizeY])
  return resized_image, label

def myDatasetToNdarray(
    train_data: "DatasetV1Adapter", test_data: "DatasetV1Adapter"
) -> typing.Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:

    preprocessed_train_data = train_data.take(n_train).map(myPreprocess).prefetch(2)
    preprocessed_test_data = test_data.take(n_test).map(myPreprocess).prefetch(2)

    print("Creating numpy arrays for image data...")

    x = np.empty([n_train, img_sizeX, img_sizeY, 3], dtype="float32")
    y = np.empty([n_train], dtype="int64")
    x_test = np.empty([n_test, img_sizeX, img_sizeY, 3], dtype="float32")
    y_test = np.empty([n_test], dtype="int64")

    print("Train data: conversion to numpy...")

    i = 0;
    for elem in preprocessed_train_data:
        x[i] = elem[0].numpy()
        y[i] = elem[1].numpy()
        i += 1
        if i % 100 == 0 or i == n_train: print('\r', i, end='')

    print("\nTest data: conversion to numpy...") 

    i = 0;
    for elem in preprocessed_test_data:
        x_test[i] = elem[0].numpy()
        y_test[i] = elem[1].numpy()
        i += 1
        if i % 100 == 0 or i == n_test: print('\r', i, end='')

    print("\nDone.")
    return x, y, x_test, y_test

# Durchmischen der Trainingsdaten, dass nicht mit sortierten Bildern trainiert wird 
train_data = train_data.shuffle(10000)

# convert tf.dataset to numpy array
X, y, X_test, y_test = myDatasetToNdarray(train_data, test_data)

#memory saving
del train_data
del test_data


n_classes = len(classes)
dataset_size = len(X)
batch_size = 32

Creating numpy arrays for image data...
Train data: conversion to numpy...


Defining the model via function

In [0]:
# returns a compiled keras model with the parameters given
def wrap_model(
    learning_rate=0.001, 
    momentum=0.9,
    decay=0.001, 
    dense_neurons=1000,
    n_filters=32,
    first_kernel_size=(7,7)):

  activation='elu'

  input_layer = Input(shape=(224, 224, 3))

  model = Conv2D(
      filters=n_filters, 
      kernel_size=(7,7), 
      activation=activation
  )(input_layer)
  model = MaxPooling2D((2,2))(model)

  for i in range(2, 5):
    model = Conv2D(
        filters = i * n_filters, 
        kernel_size=(3,3), 
        activation=activation
      )(model)
    model = MaxPooling2D((2,2))(model)

  model = Conv2D(
      filters = 5 * n_filters, 
      kernel_size=(3,3), 
      activation=activation, 
      padding='same'
    )(model)
  model = MaxPooling2D((2,2))(model)

  model = Flatten()(model)
  model = Dense(dense_neurons, activation=activation)(model)
  # model = Dense(dense_neurons / 2, activation='tanh')(model)
  model = Dense(dense_neurons / 2, activation='tanh')(model)

  output = Dense(n_classes, activation="softmax")(model)

  CNN_model = Model(input_layer, output)

  optimizer = keras.optimizers.SGD(
      lr=learning_rate, 
      momentum=momentum, 
      decay=decay
  )
  CNN_model.compile(
      loss="sparse_categorical_crossentropy", 
      optimizer=optimizer,
      metrics=["accuracy"]
  )
  
  return CNN_model

In [0]:
CNN_model = wrap_model()
CNN_model.summary()

Randomized Search

In [0]:
params_random = {
    'learning_rate':[0.01, 0.03], 
    'momentum':[0.9, 0.99, 0.999],
    'decay':[0.1, 0.01, 0.001], 
    'dense_neurons':[200, 500],# 1000, 2000],
    'n_filters':[16, 32, 64],
    'first_kernel_size':[(3,3), (5,5), (7,7)]
}

In [5]:
keras_regressor = tf.keras.wrappers.scikit_learn.KerasRegressor(wrap_model)
random_search = RandomizedSearchCV(keras_regressor, params_random, n_iter=20, verbose=1)
random_search.fit(X, y, epochs=12, validation_data=(X_test, y_test))
print("Best Parameters: ", random_search.best_params_)
best_model = random_search.best_estimator_
from deeplearning2020 import Submission
Submission('4d712a6d0ae14f2a395c992ad3627476', '3', best_model).submit()

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
NameError: name 'n_classes' is not defined

  FitFailedWarning)
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  5.1min finished


NameError: ignored